In [ ]:
from read_edf import plot_signals
from read_result import read_result_file
from read_result import plot_sleep_stages
from split_signal import split_edf_by_annotations
from split_signal import split_edf_by_30s
from sklearn.decomposition import PCA
import processing_data
import os
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# edf_path = "../database/sleep-telemetry/ST7011J0-PSG.edf"
# result_path = "../database/sleep-telemetry/ST7011JP-Hypnogram.edf"

In [ ]:
edf_files = []
result_files = []

for dirname, _, filenames in os.walk('../database/sleep-cassette'):
    for filename in filenames:
        if filename[9] == 'P':
            edf_files.append(os.path.join(dirname, filename))
        else:
            result_files.append(os.path.join(dirname, filename))
N = len(result_files)
X = np.empty((0,18000), float)
Y = np.empty((0,1), str)

print(N)

for i in range(N):
    result_path = result_files[i]
    edf_path = edf_files[i]
    # print(edf_path)
    # print(result_path)
    start_times, durations, labels = read_result_file(result_path)
    # print(durations)
    # print(labels)
    res = split_edf_by_annotations(edf_path, start_times, durations, labels)
    # print(res)
    split_signals = split_edf_by_30s(res)
    for (_, signals) in split_signals:
        for i in range(3):
            signals[i] = processing_data.signal_filter(signals[i])
    cur_X, cur_Y = processing_data.generate_matrix(split_signals)
    X = np.vstack((cur_X, X))
    Y = np.vstack((cur_Y, Y))
    # print(cur_X.shape)
    # print(cur_Y.shape)
    # break
print(X.shape)
print(Y.shape)

In [ ]:
pca1 = PCA(n_components= 200).fit(X.T)
pca2 = PCA(n_components= 0.95).fit(X.T)
x_pca1 = pca1.transform(X.T)
x_pca2 = pca2.transform(X.T)
print(x_pca1.shape)
print(x_pca2.shape)